In [113]:
from PIL import Image
import numpy as np
import copy
# import cv2
import os
from scipy.cluster.vq import whiten
import matplotlib.pyplot as plt
from sklearn.cluster import AgglomerativeClustering
from sklearn.cluster import KMeans
from sklearn.manifold import TSNE
import warnings
from sklearn.decomposition import PCA
warnings.filterwarnings("ignore")
import pandas as pd
from scipy.sparse import csr_matrix, hstack
import matplotlib.pyplot as plt
import seaborn as seab
from numpy import histogram
import warnings
from sklearn.model_selection import KFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC,LinearSVC
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB, GaussianNB
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score
from sklearn.utils import shuffle
from sklearn.metrics import matthews_corrcoef as mcc
from sklearn.metrics import classification_report
# from xgboost import XGBClassifier
warnings.filterwarnings("ignore")
import scipy
from scipy.misc import imread
# from skimage.color import rgb2lab
# from skimage.color import rgb2gray
# from skimage.measure import regionprops
import pickle
import random
import seaborn as sb
# from skimage.feature import hog,local_binary_pattern
from sklearn.model_selection import train_test_split
# from skimage import data, exposure
# train_test_split_ratio = 0.7

In [2]:
from feature_selection_ga import FeatureSelectionGA

In [3]:
# !pip3 install seaborn

# Dataset 2 Read

In [4]:
data2 = np.genfromtxt("./breast-cancer-wisconsin.data",delimiter=",")

In [5]:
train_data2 = data2.T[1:].T
train_X2 = train_data2.T[0:-1].T.astype(float)
train_Y2 = train_data2.T[-1].T
for i in range(len(train_Y2)):
    if(train_Y2[i] == 2):
        train_Y2[i] = 1
    else:
        train_Y2[i] = 0
# print (train_Y2)
# print len(train_X2)
# print len(train_X2[0])
train_X2 = np.array(train_X2)
train_Y2 = np.array(train_Y2)
train_X2,train_Y2 = shuffle(train_X2,train_Y2)

# Dataset 1 read

In [6]:
completedata = pd.read_csv("./data/data.csv")
completedata.head()
completedata.loc[completedata.diagnosis == 'M', 'diagnosis'] = 1
completedata.loc[completedata.diagnosis == 'B', 'diagnosis'] = 0

In [7]:
train_Y = completedata.diagnosis
train_X = completedata.drop(["id","Unnamed: 32","diagnosis"],axis=1)

In [8]:
train_Y = np.array(train_Y)
train_X = np.array(train_X)
train_X,train_Y = shuffle(train_X,train_Y)

In [9]:
# train_X.iloc()

In [10]:
# print (train_X.shape)
# # print (completedata.shape)
# print (train_X[0:3,:])

In [11]:
# train_X.head()

In [12]:
train_Y.shape

(569,)

In [13]:
# mask = np.zeros_like(train_X.corr(), dtype=np.bool)
# mask[np.triu_indices_from(mask)] = True
# plt.figure(figsize=(16,8))
# sb.heatmap(np.abs(train_X.corr()),cmap='YlGn',mask=mask, annot=True)
# plt.show()

In [14]:
# ax = sb.countplot(train_Y,label="Count")       # M = 212, B = 357
# B, M = train_Y.value_counts()
# print('Number of Benign: ',B)
# print('Number of Malignant : ',M)

# Dataset read complete

In [43]:
import rotation_forest
def reports(classifier,train_data,train_labels,train_test_split_ratio=.2):
    scores = []
    X_train, X_test, y_train, y_test = train_test_split(train_data, train_labels,shuffle=False, test_size=train_test_split_ratio)
    kf = KFold(n_splits=5,shuffle=False)
    kf.get_n_splits(X_train)
    print(kf)
    scores = []
    for train_index, test_index in kf.split(X_train):
        #print("TRAIN:", len(train_index), "TEST:", len(test_index))
        X_traini, X_testi = X_train[train_index], X_train[test_index]
        y_traini, y_testi = y_train[train_index], y_train[test_index]
        classifier.fit(X_traini,y_traini)
        predicted = classifier.predict(X_testi)
        scores.append(accuracy_score(predicted,y_testi))
    scores = np.array(scores)
    print ("Average Accuracy K Fold: ",scores.mean())
    
    classifier.fit(X_train,y_train)
    predicted = classifier.predict(X_test)
    print ("Test Data Results:")
    print ("Test Accuracy: ",accuracy_score(predicted,y_test))
    X = classification_report(y_test,predicted)
    print (X)
#     print ("MCC: ",mcc(test_y,predicted))
#     print ("")

In [17]:
from sklearn.decomposition import PCA
pca = PCA(n_components=25)

In [19]:
reports(rotationforest,np.array(train_X),np.array(train_Y),0.2)

KFold(n_splits=5, random_state=None, shuffle=False)


IndexError: index 364 is out of bounds for axis 0 with size 364

In [55]:
train_Y[0:5]

array([1, 1, 1, 1, 1])

In [175]:
ga = FeatureSelectionGA(rotationforest,train_X,train_Y,cv_split=10)
pop = ga.generate(100)

EVOLVING.......
-- GENERATION 1 --
Evaluated 56 individuals
-- GENERATION 2 --
Evaluated 52 individuals
-- GENERATION 3 --
Evaluated 65 individuals
-- GENERATION 4 --
Evaluated 55 individuals
-- GENERATION 5 --
Evaluated 54 individuals
-- Only the fittest survives --
Best individual is [1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1], (0.964200477326969,)


In [174]:
print (pop)

[[0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0], [0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0], [1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1], [1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0], [1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1], [0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0], [0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1], [1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1], [0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0], [1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1], [0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 

In [112]:
# print data2[0]

In [125]:
# train_data2 = data2.T[1:].T
# train_X2 = []
# train_Y2 = []

# for i in range(len(train_data2)):
#     if(np.nan not in train_data2[i]):
#         train_X2.append(train_data2[i][0:-1])
#         if(train_data2[i][-1] == 2):
#             train_Y2.append(1)
#         else:
#             train_Y2.append(0)

# # train_Y2 = train_data2.T[-1].T
# # for i in range(len(train_Y2)):
# #     if(train_Y2[i] == 2):
# #         train_Y2[i] = 1
# #     else:
# #         train_Y2[i] = 0
# print (train_Y2)
# print len(train_X2)
# # print len(train_X2[0])

In [164]:
ga = FeatureSelectionGA(rotationforest,train_X2,train_Y2,cv_split=10)
pop = ga.generate(50,mutxpb=0.4,ngen=6,cxpb=0.6)

EVOLVING.......
-- GENERATION 1 --
Evaluated 43 individuals
-- GENERATION 2 --
Evaluated 40 individuals
-- GENERATION 3 --
Evaluated 38 individuals
-- GENERATION 4 --
Evaluated 42 individuals
-- GENERATION 5 --
Evaluated 40 individuals
-- GENERATION 6 --
Evaluated 40 individuals
-- Only the fittest survives --
Best individual is [1, 0, 1, 0, 1, 1, 0, 1, 1], (0.9817767653758542,)


In [129]:
rotationforest.fit(train_X2,train_Y2)

RotationForestClassifier(bootstrap=False, class_weight=None, criterion='gini',
             max_depth=None, max_features=1.0, max_leaf_nodes=None,
             min_samples_leaf=1, min_samples_split=2,
             min_weight_fraction_leaf=0.0, n_estimators=10,
             n_features_per_subset=3, n_jobs=1, oob_score=False,
             random_state=None, rotation_algo='pca', verbose=0,
             warm_start=False)

In [130]:
y = rotationforest.predict(train_X2)

In [132]:
accuracy_score(y_pred=y,y_true=train_Y2)

1.0

In [168]:
rotationforest = rotation_forest.RotationForestClassifier(random_state=False,max_depth=4,n_estimators=10)
reports(rotationforest,train_X,train_Y,0.2)

KFold(n_splits=5, random_state=None, shuffle=False)
('Average Accuracy K Fold: ', 0.945054945054945)
Test Data Results:
('Test Accuracy: ', 0.9736842105263158)
              precision    recall  f1-score   support

           0       0.99      0.97      0.98        72
           1       0.95      0.98      0.96        42

   micro avg       0.97      0.97      0.97       114
   macro avg       0.97      0.97      0.97       114
weighted avg       0.97      0.97      0.97       114



In [107]:
rotationforest = rotation_forest.RotationForestClassifier(random_state=False,max_depth=4,n_estimators=10)
reports(rotationforest,train_X2,train_Y2,0.2)

KFold(n_splits=5, random_state=None, shuffle=False)
('Average Accuracy K Fold: ', 0.9688407005838199)
Test Data Results:
('Test Accuracy: ', 0.9781021897810219)
              precision    recall  f1-score   support

         0.0       0.95      1.00      0.97        55
         1.0       1.00      0.96      0.98        82

   micro avg       0.98      0.98      0.98       137
   macro avg       0.97      0.98      0.98       137
weighted avg       0.98      0.98      0.98       137



In [23]:
print len(train_X2)

683


In [121]:
logistic = LogisticRegression(random_state=0, solver='lbfgs',multi_class='multinomial')
reports(logistic,train_X2,train_Y2,0.2)

KFold(n_splits=5, random_state=None, shuffle=False)
('Average Accuracy K Fold: ', 0.9670058381984988)
Test Data Results:
('Test Accuracy: ', 0.9708029197080292)
              precision    recall  f1-score   support

         0.0       0.96      0.96      0.96        55
         1.0       0.98      0.98      0.98        82

   micro avg       0.97      0.97      0.97       137
   macro avg       0.97      0.97      0.97       137
weighted avg       0.97      0.97      0.97       137



In [144]:
from sklearn.tree import DecisionTreeClassifier
decisiontree = DecisionTreeClassifier(random_state=0,max_depth=4)
reports(decisiontree,train_X2,train_Y2,0.2)

KFold(n_splits=5, random_state=None, shuffle=False)
('Average Accuracy K Fold: ', 0.9541951626355296)
Test Data Results:
('Test Accuracy: ', 0.9197080291970803)
              precision    recall  f1-score   support

         0.0       0.89      0.91      0.90        55
         1.0       0.94      0.93      0.93        82

   micro avg       0.92      0.92      0.92       137
   macro avg       0.92      0.92      0.92       137
weighted avg       0.92      0.92      0.92       137



In [136]:
ann = MLPClassifier(solver='lbfgs', alpha=1e-5,hidden_layer_sizes=(5, 2), random_state=1)
reports(ann,train_X2,train_Y2,0.2)

KFold(n_splits=5, random_state=None, shuffle=False)
('Average Accuracy K Fold: ', 0.9578648874061718)
Test Data Results:
('Test Accuracy: ', 0.9562043795620438)
              precision    recall  f1-score   support

         0.0       0.96      0.93      0.94        55
         1.0       0.95      0.98      0.96        82

   micro avg       0.96      0.96      0.96       137
   macro avg       0.96      0.95      0.95       137
weighted avg       0.96      0.96      0.96       137



In [136]:
rbfn
reports(rbfn,train_X2,train_Y2,0.2)

KFold(n_splits=5, random_state=None, shuffle=False)
('Average Accuracy K Fold: ', 0.9578648874061718)
Test Data Results:
('Test Accuracy: ', 0.9562043795620438)
              precision    recall  f1-score   support

         0.0       0.96      0.93      0.94        55
         1.0       0.95      0.98      0.96        82

   micro avg       0.96      0.96      0.96       137
   macro avg       0.96      0.95      0.95       137
weighted avg       0.96      0.96      0.96       137



In [129]:
randomforest = RandomForestClassifier(n_estimators=10, max_depth=4,random_state=0)
reports(randomforest,train_X2,train_Y2,0.2)

KFold(n_splits=5, random_state=None, shuffle=False)
('Average Accuracy K Fold: ', 0.9706922435362803)
Test Data Results:
('Test Accuracy: ', 0.9708029197080292)
              precision    recall  f1-score   support

         0.0       0.93      1.00      0.96        55
         1.0       1.00      0.95      0.97        82

   micro avg       0.97      0.97      0.97       137
   macro avg       0.97      0.98      0.97       137
weighted avg       0.97      0.97      0.97       137



In [150]:
gausian = GaussianNB()
reports(gausian,train_X2,train_Y2,0.2)

KFold(n_splits=5, random_state=None, shuffle=False)
('Average Accuracy K Fold: ', 0.9670058381984987)
Test Data Results:
('Test Accuracy: ', 0.948905109489051)
              precision    recall  f1-score   support

         0.0       0.90      0.98      0.94        55
         1.0       0.99      0.93      0.96        82

   micro avg       0.95      0.95      0.95       137
   macro avg       0.94      0.95      0.95       137
weighted avg       0.95      0.95      0.95       137



In [143]:
svm = SVC(gamma='auto',kernel='poly')
reports(svm,train_X2,train_Y2,0.2)

KFold(n_splits=5, random_state=None, shuffle=False)
('Average Accuracy K Fold: ', 0.9414011676396997)
Test Data Results:
('Test Accuracy: ', 0.9124087591240876)
              precision    recall  f1-score   support

         0.0       0.92      0.85      0.89        55
         1.0       0.91      0.95      0.93        82

   micro avg       0.91      0.91      0.91       137
   macro avg       0.91      0.90      0.91       137
weighted avg       0.91      0.91      0.91       137

